In [3]:
from moomoo import *

In [13]:
from moomoo import *
import time
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

print('current subscription status :', quote_ctx.query_subscription())  # 查询初始订阅状态
ret_sub, err_message = quote_ctx.subscribe(['US.NVDA'], [SubType.QUOTE, SubType.TICKER], subscribe_push=False)
# 先订阅了 QUOTE 和 TICKER 两个类型。订阅成功后 OpenD 将持续收到服务器的推送，False 代表暂时不需要推送给脚本
if ret_sub == RET_OK:   # 订阅成功
    print('subscribe successfully！current subscription status :', quote_ctx.query_subscription())  # 订阅成功后查询订阅状态
    time.sleep(60)  # 订阅之后至少1分钟才能取消订阅
    ret_unsub, err_message_unsub = quote_ctx.unsubscribe(['HK.00700'], [SubType.QUOTE])
    if ret_unsub == RET_OK:
        print('unsubscribe successfully！current subscription status:', quote_ctx.query_subscription())  # 取消订阅后查询订阅状态
    else:
        print('unsubscription failed！', err_message_unsub)
else:
    print('subscription failed', err_message)
quote_ctx.close() # 结束后记得关闭当条连接，防止连接条数用尽

2024-05-29 23:30:37,421 | 27213 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=10, host=127.0.0.1, port=11111, user_id=101791627
current subscription status : (0, {'total_used': 0, 'remain': 300, 'own_used': 0, 'sub_list': {}})
subscription failed No right to subscribe NVDA


2024-05-29 23:30:37,426 | 27213 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=10
2024-05-30 00:19:01,837 | 27213 | [open_context_base.py] on_disconnect:386: Disconnected: conn_id=9 reason=CloseReason.RemoteClose msg=
2024-05-30 00:19:01,840 | 27213 | [open_context_base.py] _wait_reconnect:482: Wait reconnect in 8 seconds: host=127.0.0.1; port=11111;


In [12]:
from moomoo import *
import time
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

print('current subscription status :', quote_ctx.query_subscription())  # 查询初始订阅状态
#ret_sub, err_message = quote_ctx.subscribe(['HK.00700'], [SubType.QUOTE, SubType.TICKER], subscribe_push=False)

2024-05-29 23:29:16,001 | 27213 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=9, host=127.0.0.1, port=11111, user_id=101791627
current subscription status : (0, {'total_used': 0, 'remain': 300, 'own_used': 0, 'sub_list': {}})


In [10]:
import time
from moomoo import *
class OrderBookTest(OrderBookHandlerBase):
    def on_recv_rsp(self, rsp_pb):
        ret_code, data = super(OrderBookTest,self).on_recv_rsp(rsp_pb)
        if ret_code != RET_OK:
            print("OrderBookTest: error, msg: %s" % data)
            return RET_ERROR, data
        print("OrderBookTest ", data) # OrderBookTest 自己的处理逻辑
        return RET_OK, data
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
handler = OrderBookTest()
quote_ctx.set_handler(handler)  # 设置实时摆盘回调
quote_ctx.subscribe(['HK.00700'], [SubType.ORDER_BOOK])  # 订阅买卖摆盘类型，OpenD 开始持续收到服务器的推送
time.sleep(15)  #  设置脚本接收 OpenD 的推送持续时间为15秒
quote_ctx.close()  # 关闭当条连接，OpenD 会在1分钟后自动取消相应股票相应类型的订阅

2024-05-29 23:21:16,711 | 27213 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=7, host=127.0.0.1, port=11111, user_id=101791627
2024-05-29 23:21:31,722 | 27213 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=7


In [11]:
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)  # 创建行情对象
print(quote_ctx.get_market_snapshot('US.NVDA'))  # 获取港股 HK.00700 的快照数据
quote_ctx.close() # 关闭对象，防止连接条数用尽

2024-05-29 23:26:46,969 | 27213 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=8, host=127.0.0.1, port=11111, user_id=101791627
(-1, 'No right to get this quote')
2024-05-29 23:26:46,971 | 27213 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=8


In [9]:
trd_ctx = OpenSecTradeContext(host='127.0.0.1', port=11111)  # 创建交易对象
print(trd_ctx.place_order(price=500.0, qty=100, code="HK.00700", trd_side=TrdSide.BUY, trd_env=TrdEnv.SIMULATE))  # 模拟交易，下单（如果是真实环境交易，在此之前需要先解锁交易密码）

trd_ctx.close()  # 关闭对象，防止连接条数用尽

2024-05-29 23:05:18,468 | 27213 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=6, host=127.0.0.1, port=11111, user_id=101791627
(0,        code stock_name trd_side order_type order_status order_id    qty  \
0  HK.00700    TENCENT      BUY     NORMAL   SUBMITTING  2215294  100.0   

   price          create_time         updated_time  ...  dealt_avg_price  \
0  500.0  2024-05-29 23:05:18  2024-05-29 23:05:18  ...              0.0   

   last_err_msg remark time_in_force fill_outside_rth aux_price trail_type  \
0                                DAY              N/A       N/A        N/A   

  trail_value trail_spread currency  
0         N/A          N/A      HKD  

[1 rows x 21 columns])
2024-05-29 23:05:18,556 | 27213 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=6
